In [42]:
"""
Trabalho computacional sobre algoritmos evolutivos para solução de sistemas de equações não lineares 
"""

'\nTrabalho computacional sobre algoritmos evolutivos para solução de sistemas de equações não lineares \n'

In [43]:
"""
Sistema de equações não lineares
"""
import numpy as np
import random
# evolution strategy (mu, lambda) of the ackley objective function
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed



class EquationSystem:
    def __init__(self, equations):
        self.equations = equations

    def evaluate(self, x):
        return [np.abs(equation(x)) for equation in self.equations]


p1 = EquationSystem([
    lambda x: 0.8*(x[0]**2 + x[0] -1)*x[2] + 0.12*x[0]**2 + 2.16*x[0] - 0.12,
    lambda x: (1 + x[0]**2)*x[3] + 0.4*x[0]**2 - 1.6*x[0] - 0.4,
    lambda x: (1 + x[0]**2)*x[4] + x[0]**2 - 1,
    lambda x: (1 + x[0]**2)*x[5] + 0.8*(x[0]**2 + x[0] -1),
    lambda x: x[4]*x[6] - 0.02*x[5] - x[4] - x[2]*x[3] - 0.16*x[0],
    lambda x: x[6]**2 - 2*x[3]*x[6] + x[5]**2 + x[3]**2 - x[1]**2,
    lambda x: x[7] - x[1]*x[2],
    lambda x: 0.0476*x[2]*x[7]**12 + x[2] - 2.104
])

# p1 bounds
p1_lb = [-3, -1, -2, -1, -1, -0.5, -1.5, -1.5]
p1_ub = [1, 1, 2, 1, 1, 0.5, 1.5, 1.5]

In [44]:
p2 = EquationSystem([
    lambda x: x[4] + x[3] - 1.803,
    lambda x: (x[1] + x[2])*x[4] + 6.19116*x[3] - 1.803*(1.497 + 0.035),
    lambda x: x[5] + x[3] -0.328,
    lambda x: 0.28801*x[5] - x[1]*x[2]*x[4],
    lambda x: (-6.19116*x[0] + x[0]*x[2] + x[1]*x[4] - x[2]*x[4])*x[5] + x[0]*x[2]*x[4],
    lambda x: 1.571*x[6] + x[3] -1.803,
    lambda x: x[7] - 0.000856*x[6]**2,
    lambda x: (x[4] -x[0])*x[8] - x[0]*x[4],
    lambda x: x[8] - 377*x[1]*x[7]
])

# p2 bounds
p2_lb = [-0.5, -1, -1, -1, 1, -1, -1, 0, -1]
p2_ub = [0.5, 1, 1, 1, 2, 1, 1, 1, 1]

In [45]:
# Real-Coded Genetic Algorithm

# seed the pseudorandom number generator
seed(1)

def generate_individual(upper_bound: list, lower_bound: list) -> np.array:
    return np.array(
        [
            random.uniform(lower_bound[i], upper_bound[i])
            for i in range(len(upper_bound))
        ]
    )


def generate_population(upper_bound: list, lower_bound: list, pop_size) -> list:
    return [generate_individual(upper_bound, lower_bound) for _ in range(pop_size)]


# evaluate Using the penalty method
def evaluate_penalty(individual, equation_system):
    result = equation_system.evaluate(individual)
    penalty_sum = 0
    for r in result:
        penalty_sum += max(0, r) ** 2
    return penalty_sum

"""
 Um exemplo específico para o caso de codificação em ponto
flutuante é o chamado crossover aritmético (MICHALEWICZ, 1996). Este operador é
definido como uma fusão de dois vetores (cromossomos): se x1 e x2 são dois indivíduos
selecionados para crossover, os dois filhos resultantes serão x_1^line = alpha*x_1 + (1 - alpha)*x_2 e
x_2^line = (1 - alpha)*x_1 + alpha*x_2, sendo alpha um número aleatório pertencente ao intervalo [0, 1]. 
"""

def arithmetical_crossover(parent1, parent2, alpha):
    parent1, _ = parent1
    parent2, _ = parent2
    offspring1 = alpha * parent1 + (1 - alpha) * parent2
    offspring2 = (1 - alpha) * parent1 + alpha * parent2
    return (offspring1, None), (offspring2, None)

"""
o caso de problemas com codificação em ponto flutuante, o operador de mutação
mais popular é a mutação gaussiana (MICHALEWICZ & S CHOENAUER , 1996), modificando
todos os componentes de um cromossomo x = [x1 … xn] na forma:
x^line = x + N(0, sigma)

Sendo N(0, sigma) um vetor de variáveis aleatórias independentes, com distribuição normal,
média zero e desvio padrão sigma
"""

def gaussian_mutation(individual, sigma):
    ind, fitness = individual
    return ind + np.random.normal(0, sigma, len(ind)), fitness


def roulette_wheel_selection(evaluated_population):
    total_fitness = 0
    for individual, fitness in evaluated_population:
        total_fitness += 1/fitness
    r = random.uniform(0, total_fitness)
    acc = 0
    for individual, fitness in evaluated_population:
        acc += 1/fitness
        if acc >= r:
            return individual, fitness
    return evaluated_population[-1]


def real_coded_genetic_algorithm(equation_system, lower_bound, upper_bound, pop_size, max_gen, pc, pm, alpha, sigma, elitisim=True):
    """
    Real-Coded Genetic Algorithm
    :param equation_system: EquationSystem
    :param lower_bound: list
    :param upper_bound: list
    :param pop_size: int
    :param max_gen: int
    :param pc: float - Crossover probability
    :param pm: float - Mutation probability
    :param alpha: float - Crossover parameter
    :param sigma: float - Mutation parameter
    """
    # pop_size must be even
    if pop_size % 2 != 0:
        raise ValueError("pop_size must be even")

    population = generate_population(upper_bound, lower_bound, pop_size)
    population = [(individual, evaluate_penalty(individual, equation_system)) for individual in population]

    history = []
    for gen in range(max_gen):
        
        print(f"Generation {gen+1}", end="\r")
        # Evaluate
        evaluated_population = []
        # print(population)
        for individual, _ in population:
            fitness = evaluate_penalty(individual, equation_system)
            evaluated_population.append((individual, fitness))
        evaluated_population = sorted(evaluated_population, key=lambda x: x[1])

        # Keep 1 elite individual
        if elitisim:
            elite = evaluated_population[0]
        else:
            elite = None
        
        # Selection
        selected_population = []
        for _ in range(pop_size):
            selected_population.append(roulette_wheel_selection(evaluated_population))
        # Crossover
        offspring_population = []
        for i in range(0, pop_size, 2):
            if random.random() < pc:
                # print(selected_population[i], selected_population[i+1])
                offspring1, offspring2 = arithmetical_crossover(selected_population[i], selected_population[i+1], alpha)
                offspring_population.append(offspring1)
                offspring_population.append(offspring2)
            else:
                offspring_population.append(selected_population[i])
                offspring_population.append(selected_population[i+1])

        # Mutation
        mutated_population = []
        for individual in offspring_population:
            if random.random() < pm:
                mutated_population.append(gaussian_mutation(individual, sigma))
            else:
                mutated_population.append(individual)

        # Replace population
        population = mutated_population
        if elite:
            population[0] = elite
        
        best_fit = evaluated_population[0][1]
        history.append(best_fit)

    return evaluated_population[0], history


In [46]:
# run the genetic algorithm 30 times and collect the results
results = []
for _ in range(30):
    result, history = real_coded_genetic_algorithm(p1, p1_lb, p1_ub, 100, 2000, 0.8, 0.1, 0.5, 0.1, elitisim=True)
    best_individual, best_fitness = result
    results.append((result, history))

# statistical analysis
# taking the best, worst, mean, standard deviation and median of the results (using just the fitness values)
fitness_scores = [result[0][1] for result in results]



best = np.min(fitness_scores)
worst = np.max(fitness_scores)
mean = np.mean(fitness_scores)
std_dev = np.std(fitness_scores)
median = np.median(fitness_scores)

import pandas as pd

# create a dataframe to display the results
stats_df = pd.DataFrame({
    'Statistic': ['Best', 'Worst', 'Mean', 'Standard Deviation', 'Median'],
    'Value': [best, worst, mean, std_dev, median]
})

# import ace_tools as tools; tools.display_dataframe_to_user(name="GA Execution Statistics", dataframe=stats_df)


In [47]:
print(stats_df)

            Statistic     Value
0                Best  0.017029
1               Worst  0.093683
2                Mean  0.063002
3  Standard Deviation  0.033110
4              Median  0.078295


In [48]:
best_index = fitness_scores.index(best)
best_history = results[best_index][1]
print(best_history)

[3.290164588481514, 2.1078835285703463, 2.1078835285703463, 2.1078835285703463, 2.1078835285703463, 2.1078835285703463, 2.1078835285703463, 2.1078835285703463, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.8854445246193876, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.769313663649654, 1.7405

In [49]:
# não tá tudo dando 0 ainda, tem q melhorar o algoritmo
p1.evaluate(best_individual)

[0.09981793177438036,
 0.2133059432441493,
 0.06098364983244564,
 0.021988422690373766,
 0.1556815745438867,
 0.06555512770400584,
 0.05367739194879961,
 0.051009864394690574]

In [50]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=best_history, mode='lines'))
fig.show()

In [51]:

# penalty evaluation function
def evaluate_penalty(individual, equation_system):
    result = equation_system.evaluate(individual)
    penalty_sum = 0
    for r in result:
        penalty_sum += max(0, np.abs(r)) ** 2
    return penalty_sum

# check if a point is within the bounds of the search
def in_bounds(point, bounds):
    # enumerate all dimensions of the point
    for d in range(len(bounds)):
        # check if out of bounds for this dimension
        if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
            return False
    return True

# evolution strategy (mu, lambda) algorithm
def es_comma(objective, bounds, n_iter, step_size, mu, lam):
    best, best_eval = None, 1e+10
    # calculate the number of children per parent
    n_children = int(lam / mu)
    # initial population
    population = list()
    history = []
    for _ in range(lam):
        candidate = None
        while candidate is None or not in_bounds(candidate, bounds):
            candidate = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
        population.append(candidate)
    # perform the search
    for epoch in range(n_iter):
        # evaluate fitness for the population
        scores = [objective(c) for c in population]
        # rank scores in ascending order
        ranks = np.argsort(np.argsort(scores))
        # select the indexes for the top mu ranked solutions
        selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]
        # create children from parents
        children = list()
        for i in selected:
            # check if this parent is the best solution ever seen
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                history.append(best_eval)
                print('Epoch %d, Best: f(%s) = %.5f' % (epoch, best, best_eval))
            # create children for parent
            for _ in range(n_children):
                child = None
                while child is None or not in_bounds(child, bounds):
                    child = population[i] + randn(len(bounds)) * step_size
                children.append(child)
        # replace population with children
        population = children
    return [best, best_eval, history]


In [52]:

# define range for input
p1_lb = [-3, -1, -2, -1, -1, -0.5, -1.5, -1.5]
p1_ub = [1, 1, 2, 1, 1, 0.5, 1.5, 1.5]
# bounds = (min, max) for each variable
bounds = np.asarray([[p1_lb[i], p1_ub[i]] for i in range(len(p1_lb))])

print(bounds)
# define the total iterations
n_iter = 5000

# define the maximum step size
step_size = 0.15

# number of parents selected
mu = 20

# the number of children generated by parents
lam = 100

for _ in range(30):
    best, score, history = es_comma(lambda x: evaluate_penalty(x, p1), bounds, n_iter, step_size, mu, lam)
    # best_individual, best_fitness = result
    results.append(((best, score), history))

# perform the evolution strategy (mu, lambda) search
print('Done!')
print('f(%s) = %f' % (best, score))

# p1.evaluate(best)

# statistical analysis
# taking the best, worst, mean, standard deviation and median of the results (using just the fitness values)
fitness_scores = [result[0][1] for result in results]



best = np.min(fitness_scores)
worst = np.max(fitness_scores)
mean = np.mean(fitness_scores)
std_dev = np.std(fitness_scores)
median = np.median(fitness_scores)

# create a dataframe to display the results
stats_df = pd.DataFrame({
    'Statistic': ['Best', 'Worst', 'Mean', 'Standard Deviation', 'Median'],
    'Value': [best, worst, mean, std_dev, median]
})

best_index = fitness_scores.index(best)
best_history = results[best_index][1]
print(best_history)

# import ace_tools as tools; tools.display_dataframe_to_user(name="GA Execution Statistics", dataframe=stats_df)

[[-3.   1. ]
 [-1.   1. ]
 [-2.   2. ]
 [-1.   1. ]
 [-1.   1. ]
 [-0.5  0.5]
 [-1.5  1.5]
 [-1.5  1.5]]
Epoch 0, Best: f([-0.79726183 -0.69245955 -0.4274438   0.15019722 -0.30054683  0.32466051
 -0.8219083  -1.09548381]) = 14.17520
Epoch 0, Best: f([-0.14269685 -0.59952862  0.92915968 -0.33335373 -0.69672217  0.35309189
 -0.61885115  0.26810715]) = 9.18535
Epoch 0, Best: f([-0.03322135 -0.57063443  0.21702457  0.1557432  -0.05421448  0.47676915
  1.21806408  1.22876555]) = 7.41147
Epoch 0, Best: f([ 0.67106299  0.24348365  0.13795792  0.47934091  0.44393737 -0.31457317
  0.81020749 -0.88533567]) = 7.23654
Epoch 0, Best: f([-0.12748212  0.43790247  0.58522113 -0.40755404  0.31258043  0.41475538
  0.05628138 -1.25855952]) = 5.36017
Epoch 0, Best: f([ 0.10886681 -0.06485286  1.62348931  0.82757421  0.7983287   0.28277104
  0.9545518  -0.67718643]) = 3.86384
Epoch 1, Best: f([ 0.22300415 -0.00379758  1.54994913  0.63497611  0.80448763  0.1985065
  0.99889116 -0.86996288]) = 2.55889
Epoch 

In [53]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=best_history, mode='lines'))
fig.show()

In [58]:
for _ in range(30):
    result, history = real_coded_genetic_algorithm(p2, p2_lb, p2_ub, 100, 2000, 0.8, 0.1, 0.5, 0.1, elitisim=True)
    best_individual, best_fitness = result
    results.append((result, history))

fitness_scores = [result[0][1] for result in results]



best = np.min(fitness_scores)
worst = np.max(fitness_scores)
mean = np.mean(fitness_scores)
std_dev = np.std(fitness_scores)
median = np.median(fitness_scores)

import pandas as pd

# create a dataframe to display the results
stats_df = pd.DataFrame({
    'Statistic': ['Best', 'Worst', 'Mean', 'Standard Deviation', 'Median'],
    'Value': [best, worst, mean, std_dev, median]
})

best_index = fitness_scores.index(best)
best_history = results[best_index][1]
print(best_history)

fig = go.Figure()

fig.add_trace(go.Scatter(y=best_history, mode='lines'))
fig.show()

[7.866561872550463, 4.886004886258941, 0.16277709148010264, 0.16277709148010264, 0.09421359875698761, 0.09421359875698761, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.07818765467167901, 0.047118002973371154, 0.04558641586604434, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.04031757812038274, 0.0333139863675

In [55]:
p2.evaluate(best_individual)

[0.0076365470414443415,
 0.020291632949930083,
 0.031933890506983365,
 0.007046873808693997,
 0.018949811596049304,
 0.00253203812018743,
 0.0010510423245243014,
 0.003636642046804594,
 0.021892618146767694]

In [56]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=history, mode='lines'))
fig.show()

In [57]:
# bounds = (min, max) for each variable
bounds = np.asarray([[p2_lb[i], p2_ub[i]] for i in range(len(p2_lb))])

print(bounds)
# define the total iterations
n_iter = 5000

# define the maximum step size
step_size = 0.15

# number of parents selected
mu = 20

# the number of children generated by parents
lam = 100

# perform the evolution strategy (mu, lambda) search
best, score, history = es_comma(lambda x: evaluate_penalty(x, p2), bounds, n_iter, step_size, mu, lam)
print('Done!')
print('f(%s) = %f' % (best, score))

for _ in range(30):
    best, score, history = es_comma(lambda x: evaluate_penalty(x, p1), bounds, n_iter, step_size, mu, lam)
    # best_individual, best_fitness = result
    results.append(((best, score), history))

# perform the evolution strategy (mu, lambda) search
print('Done!')
print('f(%s) = %f' % (best, score))

# p1.evaluate(best)

# statistical analysis
# taking the best, worst, mean, standard deviation and median of the results (using just the fitness values)
fitness_scores = [result[0][1] for result in results]



best = np.min(fitness_scores)
worst = np.max(fitness_scores)
mean = np.mean(fitness_scores)
std_dev = np.std(fitness_scores)
median = np.median(fitness_scores)

# create a dataframe to display the results
stats_df = pd.DataFrame({
    'Statistic': ['Best', 'Worst', 'Mean', 'Standard Deviation', 'Median'],
    'Value': [best, worst, mean, std_dev, median]
})

best_index = fitness_scores.index(best)
best_history = results[best_index][1]
print(best_history)


import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=history, mode='lines'))
fig.show()

[[-0.5  0.5]
 [-1.   1. ]
 [-1.   1. ]
 [-1.   1. ]
 [ 1.   2. ]
 [-1.   1. ]
 [-1.   1. ]
 [ 0.   1. ]
 [-1.   1. ]]
Epoch 0, Best: f([ 0.46634001 -0.3025226  -0.38731586 -0.48868698  1.45406416  0.65068563
  0.78609375  0.10299723 -0.97817541]) = 171.38777
Epoch 0, Best: f([-0.35823251  0.11415498 -0.27119829  0.46238703  1.24598207  0.97404759
 -0.89190458  0.28478752  0.05942987]) = 166.08866
Epoch 0, Best: f([ 0.3496753  -0.07764664 -0.84251891 -0.15189333  1.19477752 -0.84585741
  0.65040813  0.25230626  0.81386652]) = 94.65086
Epoch 0, Best: f([-0.43379019  0.01265481 -0.94005725 -0.25300132  1.57404657  0.36819217
 -0.23133061  0.23590016  0.69306262]) = 49.64013
Epoch 0, Best: f([ 4.92748185e-01  1.54676239e-06  4.22667263e-02  9.01650263e-01
  1.23955804e+00 -7.73547870e-01 -1.10229814e-01  9.48694051e-01
 -5.90687993e-01]) = 17.77419
Epoch 1, Best: f([-0.08687455  0.01657998  0.1696548   0.32084293  1.95556727 -0.76777689
 -0.72466416  0.31556929 -0.46845806]) = 14.63012
Epo

[0.021545210775450796,
 0.03938558301773387,
 0.0038720835249782537,
 0.061093978602421516,
 0.042223967609483715,
 0.009119004663948438,
 0.0904674001201321,
 0.055294372009495135,
 0.02255517878485483]